In [23]:
import gensim
import numpy as np
import pandas as pd
import pysolr
from gensim.models import Doc2Vec

np.random.seed(42)
import tensorflow as tf
import sys

In [24]:
# In case your sys.path does not contain the base repo, go there.
print(sys.path)
%cd '/Users/axelsirota/repos/ml-solr-course'

['/Users/axelsirota/repos/ml-solr-course/3-query-generation/lab7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython']
/Users/axelsirota/repos/ml-solr-course


In [25]:
model_path = '3-query-generation/lab6/alternative_queries'
query = 'Midtown sunny two bedroom'
alternative_queries_model = tf.saved_model.load(model_path)
print(f'Model loaded')


Model loaded


In [26]:
# Create a client instance. The timeout and authentication options are not required.
solr = pysolr.Solr('http://localhost:8983/solr/airbnb', always_commit=True, timeout=10)

In [27]:
non_expanded_results = solr.search(query, **{
                'rows': 100,
            })

In [28]:
for result in non_expanded_results:
    print(f'The ID is {result["id"]} and title is {result["name"]}')


The ID is 10684164 and title is Private 1 Bdrm Apt - Prospect Park
The ID is 4248788 and title is W70s Lg. Studio with outdoor space
The ID is 48270841 and title is Romantic East Village Getaway
The ID is 39920510 and title is Large Elevator 1 Bedroom Best Location
The ID is 15721847 and title is Colorful Artist Loft w. your own Ocean Oasis
The ID is 30811958 and title is Sunny 2 Bedroom Penthouse with Good Vibes
The ID is 23852303 and title is A house in Flatbush.
The ID is 24894476 and title is Super Large Bedroom in Harlem
The ID is 19314589 and title is THE SMALL CHATEAU IN THE TREES
The ID is 19160912 and title is Spacious Guest room 3 blocks from NY Presbyterian!
The ID is 44752485 and title is West 37th street! Sunny, beautiful, serene 2BR
The ID is 11943 and title is Country space in the city
The ID is 11573269 and title is SUNNY STUDIO - 23rd FLOOR, DOORMAN
The ID is 11584340 and title is Super Cute and Sunny Studio
The ID is 18485487 and title is Sunny, Spacious, Private Room

In [29]:
states = None
next_char = tf.constant([query])
alternative_query = []

for n in range(75):
  next_char, states = alternative_queries_model.generate_one_step(next_char, states=states)
  alternative_query.append(next_char)

print(tf.strings.join(alternative_query)[0].numpy().decode("utf-8"))


l and coming space is lighting..There is a hostis of the dire for building 


In [30]:
new_query = ' '.join([query, ' ', tf.strings.join(alternative_query)[0].numpy().decode("utf-8")])

In [31]:
new_query

'Midtown sunny chateau l and coming space is lighting..There is a hostis of the dire for building '

In [32]:
expanded_results = solr.search(new_query, **{
                'rows': 100,
            })

In [33]:
for result in expanded_results:
    print(f'The ID is {result["id"]} and title is {result["name"]}')


The ID is 9474607 and title is Brand new apt in heart of Midtown!
The ID is 592853 and title is A GEM IN THE CITY
The ID is 13445027 and title is Large sunny Bedroom in Brooklyn Loft
The ID is 10684164 and title is Private 1 Bdrm Apt - Prospect Park
The ID is 22792711 and title is Bright and Beautiful Artists’ Haven
The ID is 40912943 and title is Fun Comfy & Cozy Railroad Apt near L train
The ID is 31556929 and title is Sunny Central Corner Room in Bushwick
The ID is 13424027 and title is LOFT-LIKE FLOOR THRU BROWNSTONE LIGHT SPACIOUS
The ID is 698094 and title is 5th Ave Apartment in Midtown!
The ID is 400039 and title is Big Beautiful Railroad in Brooklyn
The ID is 14406256 and title is Sunny bedroom, private backyard
The ID is 1404828 and title is Bohemian Paradise in Industrial BK
The ID is 28313785 and title is Luxury 2BR apartment in Midtown
The ID is 4248788 and title is W70s Lg. Studio with outdoor space
The ID is 14668738 and title is Sunny Room near Grand Central, Central Pa

In [34]:
model_path = '2-ranking/lab4/airbnb_model'
doc2vec_model = Doc2Vec.load(model_path)
print(f'Doc2Vec Model loaded')

Doc2Vec Model loaded


In [35]:
tokenized_query = list(gensim.utils.simple_preprocess(query))
tokenized_new_query = list(gensim.utils.simple_preprocess(new_query))

In [36]:
tokenized_new_query

['midtown',
 'sunny',
 'chateau',
 'and',
 'coming',
 'space',
 'is',
 'lighting',
 'there',
 'is',
 'hostis',
 'of',
 'the',
 'dire',
 'for',
 'building']

In [37]:
df_non_expanded_results = pd.DataFrame(non_expanded_results)
similarities = []
for result in non_expanded_results:
    try:
        similarity = doc2vec_model.similarity_unseen_docs(doc_words1=tokenized_query, doc_words2= list(gensim.utils.simple_preprocess(result["description"])))
    except KeyError:
        similarity = 0
    similarities.append(similarity)
df_non_expanded_results["Similarity"] = pd.Series(similarities)
df_non_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)


In [38]:
df_non_expanded_results

,id,listing_url,name,description,neighborhood_overview,host_id,host_url,host_name,neighbourhood,neighbourhood_cleansed,...,accommodates,bathrooms_text,bedrooms,beds,amenities,price,number_of_reviews,reviews_per_month,_version_,Similarity
13,11584340,https://www.airbnb.com/rooms/11584340,Super Cute and Sunny Studio,This sunny studio is located in the heart of H...,NaN,10563705,https://www.airbnb.com/users/show/10563705,Hailey,NaN,Washington Heights,...,2,1 bath,NaN,1.0,"[""Kitchen"", ""TV"", ""Carbon monoxide alarm"", ""Es...",$110.00,0,NaN,1705862867488604161,0.432699
52,982276,https://www.airbnb.com/rooms/982276,*Perfect Bedroom! Large Sunny Apt*Manhattan*NYC,"Large sunny apt in historic Chinatown, Lower M...","I live in Chinatown, Lower East Side. I love t...",1449904,https://www.airbnb.com/users/show/1449904,Edel,"New York, United States",Two Bridges,...,1,1.5 baths,1.0,1.0,"[""Refrigerator"", ""Essentials"", ""Long term stay...",$98.00,49,[0.51],1705862865376772098,0.403186
7,24894476,https://www.airbnb.com/rooms/24894476,Super Large Bedroom in Harlem,Large and sunny bedroom in Harlem. Two blocks ...,Sugar Hill is a landmark residential area in N...,15940253,https://www.airbnb.com/users/show/15940253,Raimundo,"New York, United States",Harlem,...,2,1 shared bath,1.0,1.0,"[""Shampoo"", ""Wifi"", ""Long term stays allowed"",...",$72.00,4,[0.14],1705862869734653954,0.402635
90,19206550,https://www.airbnb.com/rooms/19206550,Huge sunny room PRIVATE BATHROOM 15 min 2 Time...,Beautiful private guest room with full private...,Safe and quiet location. One block away from 7...,128156060,https://www.airbnb.com/users/show/128156060,Doa,"Woodside, New York, United States",Woodside,...,3,1 private bath,1.0,2.0,"[""Dishes and silverware"", ""Air conditioning"", ...",$65.00,22,[0.51],1705862868722778115,0.394241
16,16427245,https://www.airbnb.com/rooms/16427245,Beautiful private room by N/W subway in Astoria,"Sunny, contemporary, spacious and newly renova...",NaN,18978686,https://www.airbnb.com/users/show/18978686,Kristin,NaN,Ditmars Steinway,...,1,1 shared bath,1.0,1.0,"[""Hair dryer"", ""Shampoo"", ""Kitchen"", ""Breakfas...",$60.00,0,NaN,1705862868331659265,0.385350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,4707057,https://www.airbnb.com/rooms/4707057,"Private, sunny room!",Located in northern Harlem and close to the tr...,"Great neighborhood, quiet and friendly people....",6327629,https://www.airbnb.com/users/show/6327629,Bb,"New York, United States",Harlem,...,1,1.5 baths,1.0,1.0,"[""Kitchen"", ""Washer"", ""Dryer"", ""Wifi"", ""Long t...",$90.00,1,[0.01],1705862866169495552,0.071910
76,2142092,https://www.airbnb.com/rooms/2142092,"Furnished room - W. 181 St. by A, 1",Furnished room in renovated railroad apartment...,You can't beat Washington Heights - a neighbor...,8280182,https://www.airbnb.com/users/show/8280182,Alejandro,"New York, United States",Washington Heights,...,1,1 bath,1.0,1.0,"[""Kitchen"", ""Heating"", ""Wifi"", ""Long term stay...",$300.00,0,NaN,1705862865645207555,0.059720
87,1623431,https://www.airbnb.com/rooms/1623431,Enormous Chrysler-View Bedroom/Bath,This west-facing Chrysler-view bedroom has hig...,"Food selection, access to transportation, cent...",5440087,https://www.airbnb.com/users/show/5440087,Dn,"New York, United States",Midtown,...,2,1 bath,1.0,1.0,"[""Kitchen"", ""TV"", ""Essentials"", ""Washer"", ""Dry...",$292.00,1,[0.01],1705862865522524160,0.050045
58,29825681,https://www.airbnb.com/rooms/29825681,"Airy, Clean, Elegant, Sunny Studio in Midtown ...","Beautiful Art-Deco 24 hour doorman, elevator p...",The neighborhood of Sutton Place in Midtown Ea...,1995636,https://www.airbnb.com/users/show/1995636,Oleg,"New York, United States",Midtown,...,2,1 bath,NaN,1.0,"[""Hair dryer"", ""Kitchen"", ""Waterfront"", ""Eleva...",$225.00,0,NaN,1705862870418325504,0.049020


In [39]:
print(f'Most similar document before expansion has description: \n\n{df_non_expanded_results["description"].iloc[0]}\nWith similarity: {df_non_expanded_results["Similarity"].iloc[0]}')

Most similar document before expansion has description: 

This sunny studio is located in the heart of Hudson Heights. An amazing location in Manhattan. Very close to great restaurants, beautiful parks. Just ONE block away from the express A train you are in midtown in 20 minutes!
With similarity: 0.4326986074447632


In [44]:
df_expanded_results = pd.DataFrame(expanded_results)
new_similarities = []
for result in expanded_results:
    try:
        similarity = doc2vec_model.similarity_unseen_docs(doc_words1=tokenized_query, doc_words2= list(gensim.utils.simple_preprocess(result["description"])))
    except KeyError:
        similarity = 0
    new_similarities.append(similarity)
df_expanded_results["Similarity"] = pd.Series(new_similarities)
df_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)

In [45]:
print(f'Most similar document after expansion has description: \n\n{df_expanded_results["description"].iloc[0]}\nWith similarity: {df_expanded_results["Similarity"].iloc[0]}')

Most similar document after expansion has description: 

THE APARTMENT<br />This is a sunny bedroom with a full size bed in a beautiful 6 bedroom apartment in Bushwick. The room is upstairs, so it has some privacy. The roommates are all friendly and accommodating! They all work full time, so the apartment is usually peaceful.<br /><br />THE NEIGHBORHOOD <br />The apartment is in an "up and coming" neighborhood in Bushwick, off the Halsey J.  It's a short subway ride from Williamsburg or Manhattan. There are several restaurants in the area. Feel free to ask!
With similarity: 0.36164888739585876


In [46]:
print(f'Number of documents that surpass 0.5 similarity threshold: {len(df_expanded_results[df_expanded_results["Similarity"] >= 0.5])}')


Number of documents that surpass 0.5 similarity threshold: 0
